In [1]:
import numpy as np
import networkx as nx
import pandas as pd
from pathlib import Path

from QAOAKit import (
    beta_to_qaoa_format,
    gamma_to_qaoa_format,
    qaoa_maxcut_energy,
)


from QAOAKit.utils import (    
    load_weights_into_dataframe,
    load_weighted_results_into_dataframe
)

Hey Anthony! Here's the format I am expecting from you.

To help you understand the format, here's a little example of what it should look in the end.

In [2]:
# load example

p = 2
n = 5

folder_path = Path(f"../data/weighted_angle_dat/p={p}/")

df_weights = load_weights_into_dataframe(folder_path)
df = load_weighted_results_into_dataframe(folder_path, p, n, df_weights)

In [3]:
df.head(3)

,graph_id,weight_id,C_{true opt},C_init,C_opt,pr(max),p,beta_0/pi,beta_1/pi,gamma_0/pi,gamma_1/pi,mean(weight),std(weight),p_max,beta,gamma,theta,G
0,6,1,7.0,4.0,6.083484,0.572937,2,-0.160604,-0.121312,-0.165082,-0.244394,1.333333,0.471405,2,"[-0.1606039223419462, -0.1213122647670966]","[-0.1650815831054466, -0.2443943027879826]","[-0.1650815831054466, -0.2443943027879826, -0....","(0, 1, 2, 3, 4)"
1,6,2,9.0,5.5,8.000368,0.355304,2,-0.126209,-0.073390,-0.089952,-0.187875,1.833333,0.372678,2,"[-0.1262091729731615, -0.07338985774061331]","[-0.08995225667210661, -0.1878748256470103]","[-0.08995225667210661, -0.1878748256470103, -0...","(0, 1, 2, 3, 4)"
2,6,3,7.0,4.0,6.086405,0.406062,2,-0.147026,-0.084377,-0.120477,-0.221987,1.333333,0.471405,2,"[-0.1470264659118665, -0.08437722105660865]","[-0.120477363923487, -0.221987163783562]","[-0.120477363923487, -0.221987163783562, -0.14...","(0, 1, 2, 3, 4)"


That's a lot of columns! You don't need all of them.

## Required columns

1. `'G'` (`nx.Graph`) -- weighted NetworkX graph describing the instance
2. `'beta'` (`np.array`) -- optimal parameter beta (mixer angle) you found 
3. `'gamma'` (`np.array`) -- optimal parameter gamma (phase separator angle) you found 
4. `'C_opt'` (`float`) -- optimal energy with parameters `beta` and `gamma`

Angles should be in correct format; specifically, you should have

`C_opt == qaoa_maxcut_energy(G, beta_to_qaoa_format(beta),gamma_to_qaoa_format(gamma))`

You can check that it's correct below:

In [4]:
# build your dataframe df
# df = ...

# Check that it's a dataframe
assert isinstance(df, pd.DataFrame)

# Check that G is a graph
assert np.all(
    df.apply(
        lambda row: isinstance(row["G"],nx.Graph),
        axis=1,
    )
)

# Check that it's weighted
assert np.all(
    df.apply(
        lambda row: nx.is_weighted(row["G"]),
        axis=1,
    )
)

# Check that the angles are in correct format
assert np.all(
    np.isclose(
        df.apply(
            lambda row: qaoa_maxcut_energy(
                row["G"],
                beta_to_qaoa_format(row["beta"]),
                gamma_to_qaoa_format(row["gamma"]),
            ),
            axis=1,
        ),
        df["C_opt"],
    )
)

# All assertions above should pass! You can see that they pass for the example dataframe I loaded

Finally, save dataframe as pickle

In [5]:
df.to_pickle('instances.pickle')